In [4]:
import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

from konlpy.tag import Okt
# from konlpy.tag import Mecab

from konlpy.utils import pprint
from konlpy.tag import Okt
import torchtext 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
import pyLDAvis.sklearn

# # 시각화
# import pyLDAvis.sklearn



/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
# 코랩에 konlpy 패키지를 설치합니다.
! pip install konlpy

In [ ]:
!pip install pyLDAvis

In [ ]:
# 다운로드 받을 폴더를 준비
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
# # 데이터 다운로드 

# train_data = torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
#                                   path=os.path.join(DATA_DIR, 'train.txt'))
# test_data = torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', 
#                                   path=os.path.join(DATA_DIR, 'test.txt'))

14.6MB [00:00, 74.4MB/s]
4.89MB [00:00, 38.0MB/s]


In [37]:
train_data = pd.read_csv('/content/drive/MyDrive/data/train.txt', sep='\t')
test_data = pd.read_csv('/content/drive/MyDrive/data/test.txt', sep='\t')

In [41]:
train_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_data['document'] = train_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거


test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거

train_data.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [43]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    stopwords_removed_sentence = ' '.join(stopwords_removed_sentence)
    X_train.append(stopwords_removed_sentence)

100%|██████████| 143620/143620 [07:24<00:00, 323.09it/s]


In [46]:
X_train[:5]

['아 더빙 진짜 짜증나다 목소리',
 '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다',
 '너 무재 밓었 다그 래서 보다 추천 다',
 '교도소 이야기 구먼 솔직하다 재미 없다 평점 조정',
 '사이 몬페 그 익살스럽다 연기 돋보이다 영화 스파이더맨 에서 늙다 보이다 커스틴 던스트 너무나도 이쁘다 보이다']

In [47]:
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    stopwords_removed_sentence = ' '.join(stopwords_removed_sentence)
    X_test.append(stopwords_removed_sentence)

100%|██████████| 48389/48389 [02:35<00:00, 311.23it/s]


In [48]:
X_test[:5]

['굳다 ㅋ',
 '뭐 야 평점 나쁘다 않다 점 짜다 리 더 더욱 아니다',
 '지루하다 않다 완전 막장 임 돈 주다 보기 에는',
 '만 아니다 별 다섯 개 주다 왜 로 나오다 제 심기 불편하다',
 '음악 주가 되다 최고 음악 영화']

In [49]:
tfid = TfidfVectorizer(ngram_range = (1, 3), min_df = 10, sublinear_tf = True)

review_tfid = tfid.fit_transform(X_train)

In [50]:
vocab = tfid.vocabulary_
print(len(vocab))
index_to_word = { v:k for k, v in vocab.items() } 

20620


In [53]:
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

{4071: '더빙', 17851: '진짜', 18105: '짜증나다', 6004: '목소리', 4077: '더빙 진짜'}


In [54]:
labels = train_data['label'].values
# 로지스틱 분류 모델링 객체를 생성 
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습 
lr.fit(review_tfid, labels)

LogisticRegression()

In [56]:
same = 0
total = 0
test_review_tfid = tfid.transform(X_test)
test_pred = lr.predict(test_review_tfid)
for test_label, pred_label in zip(test_data['label'], test_pred):
  total += 1
  # print(f'테스트 라벨: {test_label}, 예상 라벨: {pred_label} = 결과: {test_label == pred_label}')
  if test_label == pred_label:
    same += 1

print((same / total) * 100, '%')

84.48614354502057 %


In [60]:
train_data_1 = pd.read_csv('/content/drive/MyDrive/data/comment/movie_comments_all_time.csv')
train_data_2 = pd.read_csv('/content/drive/MyDrive/data/comment/movie_comments_now.csv')
train_data = pd.concat([train_data_1, train_data_2])
train_data.reset_index(inplace=True)
train_data

,index,title,score,comment,user_id,date
0,0,탑건: 매버릭,10,평점 잘 안주는데 만점 줄려고 왔다,팝쟁이(rhkd****),NaN
1,1,탑건: 매버릭,10,영화관이 필요한 이유,우우훗(jong****),NaN
2,2,탑건: 매버릭,10,중국 애들은 불쌍타 나라 잘못 만나서 이런 영화를 극장에서 못보다니 ㅋㅋ,우다닷(real****),NaN
3,3,탑건: 매버릭,10,감동의눈물 나올정도입니다. 평점10점급임. 내가 전투기 탄 기분이였음,허니잼(jong****),NaN
4,4,탑건: 매버릭,10,"이 시대의 마지막 액션스타, ""톰 크루즈""",jydp****,NaN
...,...,...,...,...,...,...
1008159,91991,극장판 겁쟁이 페달,8,오.. 기대이상으로 재밌었음 ㅎㅎ,내다(gmdq****),2023.02.03 06:44
1008160,91992,극장판 겁쟁이 페달,9,겁쟁이 페달 만화 애니의 팬으로써 평점9점...연극 라디오 드라마등등 원소스 멀티유...,포대화상미륵존여래불(baks****),2023.02.03 00:48
1008161,91993,극장판 겁쟁이 페달,1,1점도 주기 아깝네요 자전거 그렇게타는거 아닌데ㅋㅋ,똥블이행님(nino****),2023.02.03 17:32
1008162,91994,극장판 겁쟁이 페달,1,자전거 그렇게 타는거 아닌데ㅋ,aa0(aa04****),2023.02.03 17:32


In [116]:
train_data.drop_duplicates(subset = ['comment'], inplace=True) # comment 열에서 중복인 내용이 있다면 중복 제거
train_data['comment'] = train_data['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_data['comment'] = train_data['comment'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['comment'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거
train_data.reset_index(inplace=True)

In [117]:
test_data.drop_duplicates(subset = ['comment'], inplace=True) # comment 열에서 중복인 내용이 있다면 중복 제거
test_data['comment'] = test_data['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['comment'] = test_data['comment'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['comment'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
test_data.reset_index(inplace=True)

In [119]:
train_data = train_data.drop(train_data[train_data['score'] >= 9].index)
train_data.reset_index(inplace=True, drop=True)
test_data = test_data.drop(test_data[test_data['score'] >= 9].index)
test_data.reset_index(inplace=True, drop=True)
train_data = train_data.loc[:, 'title':]
test_data = test_data.loc[:, 'title':]

In [126]:
test_data['score'].value_counts(normalize=True)

8    0.351129
1    0.167458
7    0.149199
6    0.135456
2    0.069407
4    0.058251
5    0.050261
3    0.018838
Name: score, dtype: float64

In [130]:
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

X_train = []
for sentence in tqdm(train_data['comment']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    stopwords_removed_sentence = ' '.join(stopwords_removed_sentence)
    X_train.append(stopwords_removed_sentence)


100%|██████████| 115310/115310 [08:54<00:00, 215.68it/s]


In [131]:
okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

X_test = []
for sentence in tqdm(test_data['comment']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    stopwords_removed_sentence = ' '.join(stopwords_removed_sentence)
    X_test.append(stopwords_removed_sentence)

100%|██████████| 25905/25905 [02:16<00:00, 190.18it/s]


In [134]:
tfid = TfidfVectorizer(ngram_range = (1, 3), min_df = 10, sublinear_tf = True)

review_tfid = tfid.fit_transform(X_train)

In [135]:
scores = train_data['score'].values
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습 
lr.fit(review_tfid, scores)

LogisticRegression()

In [137]:
test_review_tfid = tfid.transform(X_test)
print(test_review_tfid)


  (0, 16278)	0.24191886235536544
  (0, 13463)	0.2313459736788925
  (0, 11698)	0.32028394579070113
  (0, 9722)	0.20402308586499715
  (0, 8695)	0.12232780260374856
  (0, 8437)	0.28320580171610765
  (0, 8290)	0.12607450240301615
  (0, 8006)	0.21108309932441033
  (0, 7006)	0.2857646238952959
  (0, 5943)	0.2907912080462623
  (0, 5780)	0.08592006375389961
  (0, 4861)	0.1572300723605726
  (0, 4509)	0.15789570317012275
  (0, 3899)	0.33447890698105065
  (0, 3897)	0.3821701457331908
  (0, 3565)	0.12494634703787451
  (0, 689)	0.19162575589364925
  (0, 465)	0.23368541697875997
  (1, 16106)	0.19932432023303961
  (1, 15784)	0.23887596186712182
  (1, 15779)	0.17257633301563868
  (1, 15685)	0.1842670105958503
  (1, 13614)	0.20034422731482118
  (1, 13479)	0.08159355390653003
  (1, 13132)	0.20280551871460012
  :	:
  (25902, 12382)	0.42177949984228713
  (25902, 8291)	0.3897082983158065
  (25902, 8290)	0.16161258117719993
  (25902, 8273)	0.2465549580031149
  (25902, 1464)	0.25194259162204397
  (25902, 15)

In [138]:
test_pred = lr.predict(test_review_tfid)


In [144]:
same = 0
not_same = 0
for test_score, pred_score in zip(test_data['score'], test_pred):
  if -2 <= test_score - pred_score <= 2:
    same += 1
  else:
    not_same += 1 

print(same, not_same)
  # print(f'테스트 라벨: {test_score}, 예상 점수: {pred_score} = 결과: {test_score == pred_score}')

19667 6238


In [61]:
def process_df(data, column_name):
  data.drop_duplicates(subset = [column_name], inplace=True) # main_text 열에서 중복인 내용이 있다면 중복 제거
  data[column_name] = data[column_name].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
  data[column_name] = data[column_name].str.replace('^ +', "") # 공백은 empty 값으로 변경
  data[column_name].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
  data = data.dropna(how='any') # Null 값 제거
  data.reset_index(inplace=True)

  return data

In [68]:
train_data_1 = pd.read_csv('/content/drive/MyDrive/data/comment/movie_comments_all_time.csv')
train_data_2 = pd.read_csv('/content/drive/MyDrive/data/comment/movie_comments_now.csv')

train_data = pd.concat([train_data_1, train_data_2])
train_data = process_df(train_data, 'comment')
train_data

,index,title,score,comment,user_id,date
0,0,머슴 바울,10,종교계에는 정말 대단한분들이 많은듯머슴이었던 한국인 최초의 목사 뮤지컬 형식이라 더...,cij5****,2022.11.25 09:30
1,1,머슴 바울,10,김창식목사의 주를 향한 믿음의확신과 절절한사랑이 작품전체에 녹아져있습니다 그가 받은...,Qodn109(tmfr****),2022.11.27 21:56
2,2,머슴 바울,8,종교를 떠나서 아니 떠나지 않겠지만년전 몰랐던 역사를 알게되고그 시절 아무런 개인적...,뽁이(mybb****),2022.12.22 14:45
3,3,머슴 바울,8,많은 기대를 가지고 봤으나 한계도 너무 잘 보였습니다같이 간 일행들에게 미안할 정도...,한결(ccor****),2022.12.06 19:59
4,4,머슴 바울,10,뮤지컬로 노래하는 부분에서 오토튠을 입혀서 편집해보세요 보정하라는게 아님음색도 살아...,진시원(jbs1****),2022.11.27 00:29
...,...,...,...,...,...,...
55573,91991,극장판 겁쟁이 페달,8,오 기대이상으로 재밌었음 ㅎㅎ,내다(gmdq****),2023.02.03 06:44
55574,91992,극장판 겁쟁이 페달,9,겁쟁이 페달 만화 애니의 팬으로써 평점점연극 라디오 드라마등등 원소스 멀티유즈의 성...,포대화상미륵존여래불(baks****),2023.02.03 00:48
55575,91993,극장판 겁쟁이 페달,1,점도 주기 아깝네요 자전거 그렇게타는거 아닌데ㅋㅋ,똥블이행님(nino****),2023.02.03 17:32
55576,91994,극장판 겁쟁이 페달,1,자전거 그렇게 타는거 아닌데ㅋ,aa0(aa04****),2023.02.03 17:32


In [6]:
def predict_model():
    # 데이터 다운로드 
    # 다운로드 받을 폴더를 준비
    
    train_data_1 = pd.read_csv('/content/drive/MyDrive/data/comment/movie_comments_all_time.csv')
    train_data_2 = pd.read_csv('/content/drive/MyDrive/data/comment/movie_comments_now.csv')
    
    train_data = pd.concat([train_data_1, train_data_2])
    train_data = process_df(train_data, 'comment')
    
    
    okt = Okt()
    # 불용어 사전
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    X_train = []
    for sentence in tqdm(train_data['comment']):
        tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
        stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
        stopwords_removed_sentence = ' '.join(stopwords_removed_sentence)
        X_train.append(stopwords_removed_sentence)

    # TF-IDF 변환기 객체를 생성
    tfid = TfidfVectorizer()

    review_tfid = tfid.fit_transform(X_train)
    # 단어 사전 확인 (딕셔너리 형태)
    vocab = tfid.vocabulary_

    scores = train_data['score'].values
    lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습 
    lr.fit(review_tfid, scores)
    return tfid, lr

In [23]:
tfid, lr = predict_model() # comment 데이터로 학습

100%|██████████| 55578/55578 [02:37<00:00, 352.46it/s]


In [8]:
def generating_dataframe(type='review'):
    try:
        dir_path = f"/content/drive/MyDrive/data/{type}"
        for file_name in os.listdir(dir_path):
            df = pd.read_csv(os.path.join(dir_path, file_name))
            yield file_name, df
    except Exception as e:
        print(f'Error: Can\'t transform to dataframe: {e}')


In [74]:
def judge_movie(movie_name, df, tfid, lr):
  test = process_df(df, 'main_text')
  # 불용어 사전
  stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

  total = 0
  same = 0
  for index, row in test.iterrows():
    total += 1
    sentence = row['main_text']
    sentence = re.sub(r"[^가-힣]", " ", sentence)

    # 세 글자 이상의 명사만을 추출

    okt = Okt()
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    stopwords_removed_sentence = ' '.join(stopwords_removed_sentence)

    test_review_tfid = tfid.transform([stopwords_removed_sentence])

    test_pred = lr.predict(test_review_tfid)[0]
    if abs(row["star_score"] - test_pred) > 2:
      print(movie_name)
      print(f'{sentence}')
      print(f'실제 점수: {row["star_score"]}, 예상 점수: {test_pred}')

In [64]:
tfid, lr = predict_model()

100%|██████████| 55578/55578 [05:01<00:00, 184.10it/s]


In [ ]:
try:
  files = generating_dataframe()
  while True:
    file = next(files)
    judge_movie(file[0], file[1], tfid, lr)
    # print(file[0], file[1])

except StopIteration:
      print("요약 끝")

In [ ]:
def summarize_movie(name, data):

  data = process_df(data)
  
  okt = Okt()
  # 세 글자 이상의 명사를 사용 (두 글자 이하의 단어는 제거)
  cleaned_review_data = []
  review_data = data['main_text'].values


  for review in tqdm(review_data):
      tokens = okt.nouns(review)
      cleaned_tokens = [] # 세 글자 이상의 단어만 들어가는 리스트.
      for word in tokens:
          if len(word) > 2:
              cleaned_tokens.append(word)
          else:
            pass

  cleaned_review = " ".join(cleaned_tokens)
  cleaned_review_data.append(cleaned_review)

      # TF-IDF 변환기 객체를 생성
  tfid = TfidfVectorizer()

  review_tfid = tfid.fit_transform(cleaned_review_data)
  # 단어 사전 확인 (딕셔너리 형태)
  vocab = tfid.vocabulary_
  # 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
  index_to_word = { v:k for k, v in vocab.items() } 
  # LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
  lda = LatentDirichletAllocation(n_components=2)  

  # TF-IDF 벡터를 입력하여 모델 학습 
  print(name)
  lda.fit(review_tfid)
  for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])
  # LDA 토픽 모델링 결과를 시각화

  pyLDAvis.enable_notebook()
  visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
  # pyLDAvis.display(visualization)
  return visualization

In [ ]:
try:
  files = generating_dataframe()
  while True:
    file = next(files)
    visualization = summarize_movie(file[0], file[1])

    pyLDAvis.display(visualization)

except StopIteration:
      print("요약 끝")